In [4]:
import pandas as pd
import time
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, types
import pyspark.sql.functions as F
from  pyspark.sql.functions import col

spark = (
    SparkSession.builder.config("spark.sql.debug.maxToStringFields", 100)
    .appName("reviews")
    .getOrCreate()
)

data_filepath = "../data/cleaned_steam_reviews"

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 20:33:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
steam_reviews = spark.read.parquet(data_filepath)

In [6]:
matched_id = [218620, 613100, 753650, 255710, 606280, 748490, 571740, 560130, 666140, 269950, 221380, 673880, 391220, 678950, 595520, 272270, 435150, 823130, 417290, 631510, 773951, 753420, 356190, 629760, 541210, 385560, 233860, 512900, 413410, 428550, 644560, 504230, 834910, 214950, 857980, 502800, 637650, 291860, 242760, 730310, 464920, 548430, 393520, 620590, 501080, 250760, 502280, 640820, 489940, 420, 247240, 7510, 582660, 572410, 221640, 863550, 646570, 420290, 632350, 552500, 594650, 611760, 524580, 284160, 681660, 400940, 619290, 48700, 35140, 817130, 282560, 552520, 39210, 696170, 206190, 594570, 477160, 264710, 609320, 262060, 445220, 8930, 701160, 629910, 620980, 377160, 671440, 332200, 637090, 252950, 427290, 322330, 814380, 588650, 227300, 367520, 219150, 485510, 274190, 677160, 872790, 577800, 355790, 357190, 686600, 792990, 590380, 286160, 55230, 883710, 858210, 304390, 362890, 391540, 758190, 271590, 264200, 457140, 825630, 513710, 787860, 440900, 352550, 107410, 620, 626690, 339800, 585420, 285190, 524220, 582010, 570940, 723390, 240, 253230, 583950, 268910, 206440, 518790, 1055540, 418370, 638970, 728880, 252490, 447530, 698780, 238320, 544750, 258180, 32470, 105600, 200900, 517630, 772540, 899440, 782330, 4000, 424840, 537800, 239030, 420530, 535930, 292030, 583470, 421020, 704850, 690830, 275850, 213670, 247730, 242920, 508440, 428690, 543460, 593600, 294100, 613830, 581320, 268500, 736260, 412830, 204360, 322110, 671510, 598330, 312530, 527230, 289070, 447820, 841370, 732810, 70, 731490, 297130, 203160, 427520, 637670, 270880, 510510, 519860, 311690, 431960, 1056960, 113200, 555220, 374320, 712100, 481510, 225540, 673950, 578080, 212680, 413150, 812140, 530070, 760060, 219740, 47890, 381210, 40800, 250900, 945360, 543900, 306130, 240720, 526160, 205100, 677120, 621060, 312660, 382310, 646910, 238460, 644930, 466560, 503940, 779340, 425580, 788260, 688130, 447040, 288160, 476600, 8870, 495560, 493340, 546050, 283640, 337340, 275390, 281990, 364360, 413420, 341800, 323190, 960090, 214560, 236850, 574050, 359550, 433340, 460930, 569860, 648800, 379720, 379430, 390340, 429660, 405710, 683320, 787480, 582160, 636480, 589360, 346110, 648350, 250320, 239140, 489830, 292730, 394360]

# get id and game name for matched id
game_id = steam_reviews.select('app_id', 'app_name').distinct().filter(col('app_id').isin(matched_id)).orderBy('app_id')

In [9]:
# convert to list of tuple
game_id_list = list(zip(game_id.select('app_id').toPandas()['app_id'], game_id.select('app_name').toPandas()['app_name']))

In [10]:
print(game_id_list)

[('1055540', 'A Short Hike'), ('105600', 'Terraria'), ('1056960', 'Wolfenstein: Youngblood'), ('107410', 'Arma 3'), ('113200', 'The Binding of Isaac'), ('200900', 'Cave Story+'), ('203160', 'Tomb Raider'), ('204360', 'Castle Crashers'), ('205100', 'Dishonored'), ('206190', 'Gunpoint'), ('206440', 'To the Moon'), ('212680', 'FTL: Faster Than Light'), ('213670', 'South Park™: The Stick of Truth™'), ('214560', 'Mark of the Ninja'), ('214950', 'Total War: ROME II - Emperor Edition'), ('218620', 'PAYDAY 2'), ('219150', 'Hotline Miami'), ('219740', "Don't Starve"), ('221380', 'Age of Empires II (2013)'), ('221640', 'Super Hexagon'), ('225540', 'Just Cause 3'), ('227300', 'Euro Truck Simulator 2'), ('233860', 'Kenshi'), ('236850', 'Europa Universalis IV'), ('238320', 'Outlast'), ('238460', 'BattleBlock Theater'), ('239030', 'Papers, Please'), ('239140', 'Dying Light'), ('240', 'Counter-Strike: Source'), ('240', 'and 360 mm radiators'), ('240720', 'Getting Over It with Bennett Foddy'), ('24276

In [11]:
# create a dictionary with key as game_name and game_id
for i in range(len(game_id_list)):
    game_id_list[i] = {"game_id": game_id_list[i][0], "game_name": game_id_list[i][1]}


In [ ]:
game_id_list

[{'game_id': '1055540', 'game_name': 'A Short Hike'},
 {'game_id': '105600', 'game_name': 'Terraria'},
 {'game_id': '1056960', 'game_name': 'Wolfenstein: Youngblood'},
 {'game_id': '107410', 'game_name': 'Arma 3'},
 {'game_id': '113200', 'game_name': 'The Binding of Isaac'},
 {'game_id': '200900', 'game_name': 'Cave Story+'},
 {'game_id': '203160', 'game_name': 'Tomb Raider'},
 {'game_id': '204360', 'game_name': 'Castle Crashers'},
 {'game_id': '205100', 'game_name': 'Dishonored'},
 {'game_id': '206190', 'game_name': 'Gunpoint'},
 {'game_id': '206440', 'game_name': 'To the Moon'},
 {'game_id': '212680', 'game_name': 'FTL: Faster Than Light'},
 {'game_id': '213670', 'game_name': 'South Park™: The Stick of Truth™'},
 {'game_id': '214560', 'game_name': 'Mark of the Ninja'},
 {'game_id': '214950', 'game_name': 'Total War: ROME II - Emperor Edition'},
 {'game_id': '218620', 'game_name': 'PAYDAY 2'},
 {'game_id': '219150', 'game_name': 'Hotline Miami'},
 {'game_id': '219740', 'game_name': "D

23/11/30 22:15:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1809657 ms exceeds timeout 120000 ms
23/11/30 22:15:11 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/30 22:15:12 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

In [ ]:
[218620, 613100, 753650, 255710, 606280, 748490, 571740, 560130, 666140, 269950, 221380, 673880, 391220, 678950, 595520, 272270, 435150, 823130, 417290, 631510, 773951, 753420, 356190, 629760, 541210, 385560, 233860, 512900, 413410, 428550, 644560, 504230, 834910, 214950, 857980, 502800, 637650, 291860, 242760, 730310, 464920, 548430, 393520, 620590, 501080, 250760, 502280, 640820, 489940, 420, 247240, 7510, 582660, 572410, 221640, 863550, 646570, 420290, 632350, 552500, 594650, 611760, 524580, 284160, 681660, 400940, 619290, 48700, 35140, 817130, 282560, 552520, 39210, 696170, 206190, 594570, 477160, 264710, 609320, 262060, 445220, 8930, 701160, 629910, 620980, 377160, 671440, 332200, 637090, 252950, 427290, 322330, 814380, 588650, 227300, 367520, 219150, 485510, 274190, 677160, 872790, 577800, 355790, 357190, 686600, 792990, 590380, 286160, 55230, 883710, 858210, 304390, 362890, 391540, 758190, 271590, 264200, 457140, 825630, 513710, 787860, 440900, 352550, 107410, 620, 626690, 339800, 585420, 285190, 524220, 582010, 570940, 723390, 240, 253230, 583950, 268910, 206440, 518790, 1055540, 418370, 638970, 728880, 252490, 447530, 698780, 238320, 544750, 258180, 32470, 105600, 200900, 517630, 772540, 899440, 782330, 4000, 424840, 537800, 239030, 420530, 535930, 292030, 583470, 421020, 704850, 690830, 275850, 213670, 247730, 242920, 508440, 428690, 543460, 593600, 294100, 613830, 581320, 268500, 736260, 412830, 204360, 322110, 671510, 598330, 312530, 527230, 289070, 447820, 841370, 732810, 70, 731490, 297130, 203160, 427520, 637670, 270880, 510510, 519860, 311690, 431960, 1056960, 113200, 555220, 374320, 712100, 481510, 225540, 673950, 578080, 212680, 413150, 812140, 530070, 760060, 219740, 47890, 381210, 40800, 250900, 945360, 543900, 306130, 240720, 526160, 205100, 677120, 621060, 312660, 382310, 646910, 238460, 644930, 466560, 503940, 779340, 425580, 788260, 688130, 447040, 288160, 476600, 8870, 495560, 493340, 546050, 283640, 337340, 275390, 281990, 364360, 413420, 341800, 323190, 960090, 214560, 236850, 574050, 359550, 433340, 460930, 569860, 648800, 379720, 379430, 390340, 429660, 405710, 683320, 787480, 582160, 636480, 589360, 346110, 648350, 250320, 239140, 489830, 292730, 394360]

In [1]:
# read csv
import pandas as pd
df = pd.read_csv('review_counts.csv')

,app_id,app_name,review_count
0,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,1644255
1,271590,Grand Theft Auto V,1019116
2,359550,Tom Clancy's Rainbow Six Siege,841918
3,105600,Terraria,672815
4,4000,Garry's Mod,655524


In [2]:
app_id = df['app_id'].tolist()

In [3]:
print(app_id)

[578080, 271590, 359550, 105600, 4000, 252490, 252950, 218620, 945360, 292030, 381210, 346110, 227300, 413150, 72850, 431960, 582010, 367520, 242760, 322330, 374320, 620, 377160, 739630, 239140, 107410, 275850, 203160, 8930, 264710, 698780, 255710, 391540, 289070, 489830, 379720, 250900, 48700, 477160, 814380, 1145360, 240, 394360, 435150, 427520, 8870, 782330, 648800, 281990, 294100, 304390, 812140, 306130, 391220, 221380, 262060, 1289310, 646570, 552520, 236850, 594570, 219740, 883710, 629760, 225540, 524220, 594650, 204360, 379430, 548430, 270880, 268910, 518790, 397540, 779340, 238320, 588650, 581320, 582160, 440900, 552500, 268500, 644560, 284160, 219150, 457140, 323190, 960090, 238460, 55230, 433340, 212680, 70, 214950, 362890, 205100, 311690, 356190, 207610, 206440, 1057090, 508440, 460930, 546560, 113200, 863550, 1190460, 447040, 364360, 620980, 1158310, 513710, 213670, 493340, 526870, 274190, 1113000, 233860, 239030, 787860, 648350, 683320, 286160, 646910, 834910, 582660, 5709